In [1]:
import pandas as pd

# read data
reviews_df = pd.read_csv("Keru_reviews.csv")

reviews_df.head()

/home/hp/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,label,text
0,5,Looks ok. Not like so durable. Will hv to use ...
1,5,"Tried, the current can be very powerful depend..."
2,5,Item received after a week. Looks smaller than...
3,5,Thanks!!! Works as describe no complaints. Not...
4,5,Fast delivery considering it’s from overseas a...


In [2]:
reviews_df = reviews_df.sample(frac = 0.1, replace = False, random_state=42)

In [3]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
import string
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

def clean_text(text):
    # lower text
    text = text.lower()
    # tokenize text and remove puncutation
    text = [word.strip(string.punctuation) for word in text.split(" ")]
    # remove words that contain numbers
    text = [word for word in text if not any(c.isdigit() for c in word)]
    # remove stop words
    stop = stopwords.words('english')
    text = [x for x in text if x not in stop]
    # remove empty tokens
    text = [t for t in text if len(t) > 0]
    # pos tag text
    pos_tags = pos_tag(text)
    # lemmatize text
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    # remove words with only one letter
    text = [t for t in text if len(t) > 1]
    # join all
    text = " ".join(text)
    return(text)

# clean text data
reviews_df["review_clean"] = reviews_df["text"].apply(lambda x: clean_text(x))

In [4]:
# add sentiment anaylsis columns
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
reviews_df["sentiments"] = reviews_df["text"].apply(lambda x: sid.polarity_scores(x))
reviews_df = pd.concat([reviews_df.drop(['sentiments'], axis=1), reviews_df['sentiments'].apply(pd.Series)], axis=1)

In [5]:
# add number of characters column
reviews_df["nb_chars"] = reviews_df["text"].apply(lambda x: len(x))

# add number of words column
reviews_df["nb_words"] = reviews_df["text"].apply(lambda x: len(x.split(" ")))

In [6]:
reviews_df

,label,text,review_clean,neg,neu,pos,compound,nb_chars,nb_words
1121587,5,Love these masks and samples given. Will try o...,love mask sample give try sample purchase actu...,0.000,0.802,0.198,0.6369,107,18
1104606,5,Good value for 300g tube. Thank you !,good value tube thank,0.000,0.331,0.669,0.7959,37,8
339262,5,amazing!!!,amaze,0.000,0.000,1.000,0.6884,10,1
1382344,5,Ordered a Size 12 but came a Size 9 so I’m not...,order size come size i’m sure what’s go it’s okay,0.148,0.730,0.122,-0.1252,83,19
1355964,5,Ordered on 3/11/19 received item in good condi...,order received item good condition,0.000,0.632,0.368,0.6209,57,8
...,...,...,...,...,...,...,...,...,...
854047,2,Satisfied with the purchase. Soft material whi...,satisfied purchase soft material great singapo...,0.000,0.592,0.408,0.7845,81,12
747971,5,Slippers is very good quality as i expect & pe...,slipper good quality expect perfect toddler,0.000,0.566,0.434,0.7841,69,13
1387989,4,"Very pretty, although there were some threads ...",pretty although thread come received,0.000,0.775,0.225,0.5413,77,13
1489294,4,Very nice and good size bag. Delivery was a bi...,nice good size bag delivery bit slow packing w...,0.114,0.729,0.157,0.4070,175,34


In [ ]:
# add tf-idfs columns
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(min_df = 10)
tfidf_result = tfidf.fit_transform(reviews_df["review_clean"]).toarray()
tfidf_df = pd.DataFrame(tfidf_result, columns = tfidf.get_feature_names())
tfidf_df.columns = ["word_" + str(x) for x in tfidf_df.columns]
tfidf_df.index = reviews_df.index
reviews_df = pd.concat([reviews_df, tfidf_df], axis=1)